# Packaging software in images

## Building images automatically with Dockerfiles

### A Dockerfile primer

Dockerfiles are expressive and easy to understand because of their terse syntax that
allows for comments. You **can keep track of changes to Dockerfiles with any version control
system**. Maintaining multiple versions of an image is as simple as maintaining multiple Dockerfiles.

The Dockerfile build process itself uses extensive caching to aid
rapid development and iteration. The builds are traceable and reproducible. They
integrate easily with existing build systems and many continuous integration tools.
With all these reasons to prefer Dockerfile builds to handmade images, it’s important
to learn how to write them.

#### .dockerignore

The directory where you issue the docker build command is called the build context. Docker will send all of the files and directories in your build directory to the Docker daemon as part of the build context. If you have stuff in your directory that is not needed by your build, you’ll have an unnecessarily larger build context that results in a larger image size.

You can remedy this situation by adding a .dockerignore file that works similarly to .gitignore. You can specify the list of folders and files that should be ignored in the build context.

One of the best reasons to use Dockerfile builds is that they simplify copying files from your computer into an image. But it’s not always appropriate for certain files to be copied to images. The first thing to do when starting a new project is to define which files should never be copied into any images. You can do this in a file called `.dockerignore`. In this example, you’ll create three Dockerfiles, and none needs to be copied into the resulting images.

    .dockerignore
    Dockerfile

Save and close the file when you’re finished. This will prevent the .dockerignore file, or files named Dockerfile, from ever being copied into an image during a build. With that bit of accounting finished, you can begin working on the base image.

#### FROM

For beginners it’s enough to understand that **every Dockerfile must start with the FROM instruction in the form of FROM `<image>[:tag]`.** 

The tag value is optional, if **you don’t specify the tag Docker will use the tag latest and will try and use or pull the latest version of the base image during build.**


```Dockerfile

# pull official base image
FROM python:3.8.1-slim-buster
    
   ```

#### ENV

```Dockerfile
# pull official base image
FROM python:3.8.1-slim-buster

# set environment variables
ENV APPROOT="/home/app/web" \
    PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    VERSION="0.1" 
```    

>  When you create a Dockerfile, you need to keep in mind that each Dockerfile **instruction will result in a new layer being created**. Instructions should be combined whenever possible because the builder won’t perform any optimization.

#### LABEL

The LABEL instruction is used to define key/value pairs that are recorded as additional
metadata for an image or container. This mirrors the --label flag on docker run and
docker create. Like the ENV instruction before it, multiple labels can and should be
set with a single instruction. In this case, the value of the VERSION environment variable
was substituted for the value of the base.version label. By using an environment
variable in this way, the value of VERSION will be available to processes running inside a
container as well as recorded to an appropriate label. **This increases maintainability of
the Dockerfile because it’s more difficult to make inconsistent changes when the valueis
set in a single location**.

```Dockerfile
# pull official base image
FROM python:3.8.1-slim-buster

LABEL maintainer="leon@ltfe.org"

# set environment variables
ENV APPROOT="/home/app/web" \
    PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    VERSION="0.1" 

LABEL base.name="Smart API" \
      base.version="${VERSION}"


 ```   

#### EXPOSE

An important instruction to inform your users about the ports your application is listening on. EXPOSE will not publish the port, you need to use docker run -p... to do that when you start the container.

Last, the EXPOSE command creates a layer that opens TCP port 5000.

```Dockerfile
# pull official base image
FROM python:3.8.1-slim-buster

LABEL maintainer="leon@ltfe.org"

# set environment variables
ENV APPROOT="/home/app/web" \
    PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    VERSION="0.1" 

LABEL base.name="Smart API" \
      base.version="${VERSION}"

# set work directory
WORKDIR $APPROOT

EXPOSE 5000
``` 

#### RUN

RUN will execute commands, so it’s one of the most-used instructions. I would like to highlight two points:
1. You’ll use a lot of apt-get type of commands to add new packages to your image. It’s always advisable to put apt-get update and apt-get install commands on the same line. This is important because of layer caching. Having these on two separate lines would mean that if you add a new package to your install list, the layer with apt-get update will not be invalidated in the layer cache and you might end up in a mess. 
2. RUN has two forms; `RUN <command>` (called shell form) and `RUN ["executable", "param1", "param2"]` called exec form. Please note that `RUN <command>` will invoke a shell automatically (/bin/sh -c by default), while the exec form will not invoke a command shell.

```Dockerfile
# pull official base image
FROM python:3.8.1-slim-buster

LABEL maintainer="leon@ltfe.org"

# set environment variables
ENV APPROOT="/home/app/web" \
    PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    VERSION="0.1" 

LABEL base.name="Smart API" \
      base.version="${VERSION}"

# set work directory
WORKDIR $APPROOT

EXPOSE 5000

# install dependencies
RUN apt-get update && apt-get install -y --no-install-recommends netcat
RUN pip install --upgrade pip

``` 

#### COPY vs ADD

Both ADD and COPY are designed to add directories and files to your Docker image in the form of `ADD <src>... <dest>` or `COPY <src>... <dest>`. Most resources, including myself, **suggest to use COPY**.

The reason behind this is that ADD has extra features compared to COPY that make ADD more unpredictable and a bit over-designed. ADD can pull files from url sources, which COPY cannot. ADD can also extract compressed files assuming it can recognize and handle the format. You cannot extract archives with COPY.

The ADD instruction was added to Docker first, and COPY was added later to provide a straightforward, rock solid solution for copying files and directories into your container’s file system.

If you want to pull files from the web into your image I would suggest to use RUN and curl and uncompress your files with RUN and commands you would use on the command line.

```Dockerfile
# pull official base image
FROM python:3.8.1-slim-buster

LABEL maintainer="leon@ltfe.org"

# set environment variables
ENV APPROOT="/home/app/web" \
    PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    VERSION="0.1" 

LABEL base.name="Smart API" \
      base.version="${VERSION}"

# set work directory
WORKDIR $APPROOT

EXPOSE 5000

# install dependencies
RUN apt-get update && apt-get install -y --no-install-recommends netcat
RUN pip install --upgrade pip
COPY ./requirements.txt $APPROOT/requirements.txt
RUN pip install -r requirements.txt

# copy project
COPY . $APPROOT

``` 

#### USER

Don’t run your stuff as root, be humble, use the USER instruction to specify the user. This user will be used to run any subsequent RUN, CMD AND ENDPOINT instructions in your Dockerfile.

By default, Docker runs container processes as root inside of a container. This is a bad practice since attackers can gain root access to the Docker host if they manage to break out of the container. If you're root in the container, you'll be root on the host.

```Dockerfile
# pull official base image
FROM python:3.8.1-slim-buster

LABEL maintainer="leon@ltfe.org"

# set environment variables
ENV APPROOT="/home/app/web" \
    PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    VERSION="0.1" 

LABEL base.name="Smart API" \
      base.version="${VERSION}"

# create the app user
RUN groupadd -r -g 2200 app && \
    useradd -rM -g app -u 2200 app

# set work directory
WORKDIR $APPROOT

EXPOSE 5000

# install dependencies
RUN apt-get update && apt-get install -y --no-install-recommends netcat
RUN pip install --upgrade pip
COPY ./requirements.txt $APPROOT/requirements.txt
RUN pip install -r requirements.txt

# copy project
COPY . $APPROOT

# chown all the files to the app user
RUN chown -R app:app $APPROOT

# change to the app user
USER app
``` 

#### ENTRYPOINT & CMD

The ENTRYPOINT instruction has two forms: the **shell** form and an **exec** form. The
shell form looks like a shell command with whitespace-delimited arguments. The exec
form is a string array in which the first value is the command to execute and the
remaining values are arguments. A command specified using the shell form would be
executed as an argument to the default shell. Specifically, the command used in this
Dockerfile will be executed as /bin/sh –c 'exec ./mailer.sh' at runtime. Most
importantly, if the shell form is used for ENTRYPOINT, all other arguments provided by
the CMD instruction or at runtime as extra arguments to docker container run will be
ignored. This makes the shell form of ENTRYPOINT less flexible.

Naša končna slika:

```Dockerfile
# pull official base image
FROM python:3.8.1-slim-buster

LABEL maintainer="leon@ltfe.org"

# set environment variables
ENV APPROOT="/home/app/web" \
    PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    VERSION="0.1" 

LABEL base.name="Smart API" \
      base.version="${VERSION}"

# create the app user
RUN groupadd -r -g 2200 app && \
    useradd -rM -g app -u 2200 app

# set work directory
WORKDIR $APPROOT

EXPOSE 5000

# install dependencies
RUN apt-get update && apt-get install -y --no-install-recommends netcat
RUN pip install --upgrade pip
COPY ./requirements.txt $APPROOT/requirements.txt
RUN pip install -r requirements.txt

# copy project
COPY . $APPROOT

# chown all the files to the app user
RUN chown -R app:app $APPROOT

# za testne namene
RUN apt-get install -y procps

# change to the app user
USER app


ENTRYPOINT ["python", "main.py"]
CMD ["-B"]
``` 

Both CMD and ENTRYPOINT instructions define what command gets executed when running a container. There are few rules that describe their co-operation.
1. Dockerfile should specify at least one of `CMD` or `ENTRYPOINT` commands.
2. `ENTRYPOINT` should be defined when using the container as an executable.
3. `CMD` should be used as a way of defining default arguments for an `ENTRYPOINT` command or for executing an ad-hoc command in a container.
4. `CMD` will be overridden when running the container with alternative arguments.

The table below shows what command is executed for different ENTRYPOINT / CMD combinations:

<table>
  <thead>
    <tr>
      <th style="text-align: left">&nbsp;</th>
      <th style="text-align: left">No ENTRYPOINT</th>
      <th style="text-align: left">ENTRYPOINT exec_entry p1_entry</th>
      <th style="text-align: left">ENTRYPOINT [“exec_entry”, “p1_entry”]</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="text-align: left"><strong>No CMD</strong></td>
      <td style="text-align: left"><em>error, not allowed</em></td>
      <td style="text-align: left">/bin/sh -c exec_entry p1_entry</td>
      <td style="text-align: left">exec_entry p1_entry</td>
    </tr>
    <tr>
      <td style="text-align: left"><strong>CMD [“exec_cmd”, “p1_cmd”]</strong></td>
      <td style="text-align: left">exec_cmd p1_cmd</td>
      <td style="text-align: left">/bin/sh -c exec_entry p1_entry</td>
      <td style="text-align: left">exec_entry p1_entry exec_cmd p1_cmd</td>
    </tr>
    <tr>
      <td style="text-align: left"><strong>CMD [“p1_cmd”, “p2_cmd”]</strong></td>
      <td style="text-align: left">p1_cmd p2_cmd</td>
      <td style="text-align: left">/bin/sh -c exec_entry p1_entry</td>
      <td style="text-align: left">exec_entry p1_entry p1_cmd p2_cmd</td>
    </tr>
    <tr>
      <td style="text-align: left"><strong>CMD exec_cmd p1_cmd</strong></td>
      <td style="text-align: left">/bin/sh -c exec_cmd p1_cmd</td>
      <td style="text-align: left">/bin/sh -c exec_entry p1_entry</td>
      <td style="text-align: left">exec_entry p1_entry /bin/sh -c exec_cmd p1_cmd</td>
    </tr>
  </tbody>
</table>

> If CMD is defined from the base image, setting ENTRYPOINT will reset CMD to an empty value. In this scenario, CMD must be defined in the current image to have a value.


The `exec form`, which is the preferred form:
    
    ENTRYPOINT ["executable", "param1", "param2"]

The `shell form`:

    ENTRYPOINT command param1 param2

> TIP:  **Using the exec (or string array) form wherever possible is the best practice.**
At a minimum, a Dockerfile should be consistent and avoid mixing styles.
This will make your Dockerfiles more readable and ensure that instructions
behave as you’d expect without detailed understanding of their nuances.

Command line arguments to docker run <image> will be appended after all elements in an exec form ENTRYPOINT, and will override all elements specified using CMD. This allows arguments to be passed to the entry point, i.e., docker run <image> -d will pass the -d argument to the entry point. You can override the ENTRYPOINT instruction using the docker run --entrypoint flag.

    docker image build -t leon11sj/entrypointcmd:v1 -f Dockerfile_01 .

    docker run -it --rm leon11sj/entrypointcmd:v1

    docker run -it --rm leon11sj/entrypointcmd:v1 -la

    docker run -it --rm --entrypoint top leon11sj/entrypointcmd:v1

    docker run -it --rm --entrypoint ps leon11sj/entrypointcmd:v1 aux

The shell form prevents any CMD or run command line arguments from being used, but has the disadvantage that your ENTRYPOINT will be started as a subcommand of /bin/sh -c, which does not pass signals. This means that the executable will not be the container’s PID 1 - and will not receive Unix signals - so your executable will not receive a SIGTERM from docker stop <container>.

Only the last ENTRYPOINT instruction in the Dockerfile will have an effect.

    ENTRYPOINT ["python", "main.py"]

    docker image build -t leon11sj/smart-api:v0.1 -f Dockerfile.prod .
    docker run -d --name test leon11sj/smart-api:v0.1
    docker exec -it test ps x
    docker rm -f test

Druga opcija v shell obliki

    ENTRYPOINT python main.py

    docker image build -t leon11sj/smart-api:v0.1 -f Dockerfile.prod .
    docker run -d --name test leon11sj/smart-api:v0.1
    docker exec -it test ps x
    docker rm -f test

<hr>

CMD instruction allows you to set a default command, which will be executed only when you run container without specifying a command. If Docker container runs with a command, the default command will be ignored. If Dockerfile has more than one CMD instruction, all but last CMD instructions are ignored.

CMD has three forms:
- `CMD ["executable","param1","param2"]` (exec form, preferred)
- `CMD ["param1","param2"]` (sets additional default parameters for ENTRYPOINT in exec form)
- `CMD command param1 param2` (shell form)

<hr>

**The difference is ENTRYPOINT command and parameters are not ignored when Docker container runs with command line parameters.**

Exec form of ENTRYPOINT allows you to set commands and parameters and then use either form of CMD to set additional parameters that are more likely to be changed. **ENTRYPOINT arguments are always used, while CMD ones can be overwritten by command line arguments provided when Docker container runs**. For example, the following snippet in Dockerfile

    docker image build -t leon11sj/entrypointcmd:v3 -f Dockerfile_03 .

    docker run -it --rm leon11sj/entrypointcmd:v3

    docker run -it --rm leon11sj/entrypointcmd:v3 John

> Shell form of ENTRYPOINT ignores any CMD or docker run command line arguments.

<hr>

**Zaključki**:
- Prefer ENTRYPOINT to CMD when building executable Docker image and you need a command always to be executed. Additionally use CMD if you need to provide extra default arguments that could be overwritten from command line when docker container runs.
- Choose CMD if you need to provide a default command and/or arguments that can be overwritten from command line when docker container runs.

### Building the file

Put it all together by running the docker image build command from the directory where the mailer-base file is located. The -f flag tells the builder which filename to use as input:

    docker image build -t leon11sj/smart-api:v0.1 -f Dockerfile.prod .
    docker run --rm --name smart-api -p 80:5000 leon11sj/smart-api:v0.1

> **Naming Dockerfiles**: The default and most common name for a Dockerfile is Dockerfile. However, Dockerfiles
can be named anything because they are simple text files and the build command
accepts any filename you tell it. Some people name their Dockerfiles with an
extension such as .df so that they can easily define builds for multiple images in a
single project directory (for example, app-build.df, app-runtime.df, and app-debugtools.
df). A file extension also makes it easy to activate Dockerfile support in editors.

## Best practices for writing Dockerfiles

### Start your Dockerfile with the steps that are least likely to change

This is easier said than done. Anyway, your image will stabilize after a while and changes will be less likely. The best practice is to structure your Dockerfile according to the following:
1. Install tools that are needed to build your application.
2. Install dependencies, libraries and packages.
3. Build your application.

### Clean up your Dockerfile

Always review your steps in the Dockerfile and only keep the minimum set of steps that are needed by your application. Always remove unnecessary components.

### Containers should be ephemeral

This would belong to generic Docker guidelines, but it’s never enough to stress this point. It is your best interest to design and build Docker images that can be destroyed and recreated/replaced automatically or with minimal configuration.

Which means that you should create Dockerfiles that define stateless images. Any state, should be kept outside of your containers.

### One container should have one concern

Think of containers as entities that take responsibility for one aspect of your project. So design your application in a way that your web server, database, in-memory cache and other components have their own dedicated containers.

You’ll see the benefits of such a design when scaling your app horizontally. We’ll look into interoperability of containers and container networking in a future tutorial.